# Demo notebook : SelfPlay with ONNX models

## Setup

In [79]:
MODE = "local"  # "colab" | "colab-dev" | "local"

In [80]:
if MODE == "colab":
    !pip install -q lczerolens
elif MODE == "colab-dev":
    !rm -r lczerolens
    !git clone https://github.com/Xmaster6y/lczerolens -b main
    !pip install -q ./lczerolens

## Download ONNX Models

In [81]:
!gdown 1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO -O maia1100.onnx
!gdown 1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl -O maia1900.onnx

Downloading...
From: https://drive.google.com/uc?id=1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1100.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 31.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1900.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 34.1MB/s]


## Import Libraries

In [82]:
from lczerolens import LczeroBoard, LczeroModel
from lczerolens.sampling import SelfPlay, ModelSampler, MCTSSampler

import importlib
import lczerolens.sampling as sampling
import lczerolens.search as search

for module in (sampling, search):
    importlib.reload(module)

## Load & Setup Model Samplers

In [83]:
maia_1100_model = LczeroModel.from_path("maia1100.onnx")
maia_1900_model = LczeroModel.from_path("maia1900.onnx")
white_sampler = MCTSSampler(model=maia_1100_model, num_simulations=1000, use_argmax=True)
black_sampler = ModelSampler(model=maia_1900_model, use_argmax=True)

In [84]:
board = LczeroBoard()
td = maia_1100_model(board)[0]
legal_indices = board.get_legal_indices()
td["policy"] = td["policy"].gather(0, legal_indices)
# .gather(1, legal_indices).shape
td

TensorDict(
    fields={
        boards: Tensor(shape=torch.Size([112, 8, 8]), device=cpu, dtype=torch.float32, is_shared=False),
        policy: Tensor(shape=torch.Size([20]), device=cpu, dtype=torch.float32, is_shared=False),
        wdl: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([]),
    device=None,
    is_shared=False)

In [85]:
# white_sampler._heuristic.out_keys
maia_1100_model._get_output_names(maia_1100_model.module)

['policy', 'wdl']

## Initialize SelfPlay

In [86]:
self_play = SelfPlay(white=white_sampler, black=black_sampler)
logs = []


def report_fn(log, to_play):
    logs.append((log, to_play))

## Play Game

In [87]:
board = LczeroBoard()

game_moves, final_board = self_play.play(board=board, max_moves=100, report_fn=report_fn)

## Display Results

In [88]:
print("Game moves:", game_moves)
print("Game result:", final_board.result())

Game moves: [Move.from_uci('e2e4'), Move.from_uci('c7c5'), Move.from_uci('g1f3'), Move.from_uci('d7d6'), Move.from_uci('d2d4'), Move.from_uci('c5d4'), Move.from_uci('f3d4'), Move.from_uci('g8f6'), Move.from_uci('b1c3'), Move.from_uci('a7a6'), Move.from_uci('f1c4'), Move.from_uci('e7e6'), Move.from_uci('c1g5'), Move.from_uci('f8e7'), Move.from_uci('g5f6'), Move.from_uci('e7f6'), Move.from_uci('d4f3'), Move.from_uci('d8c7'), Move.from_uci('c4b3'), Move.from_uci('f6c3'), Move.from_uci('b2c3'), Move.from_uci('c7c3'), Move.from_uci('f3d2'), Move.from_uci('b8c6'), Move.from_uci('b3a4'), Move.from_uci('b7b5'), Move.from_uci('a4b3'), Move.from_uci('c8b7'), Move.from_uci('a1b1'), Move.from_uci('c6d4'), Move.from_uci('f2f3'), Move.from_uci('c3e3'), Move.from_uci('e1f1'), Move.from_uci('a8c8'), Move.from_uci('d1e1'), Move.from_uci('e3c3'), Move.from_uci('f1f2'), Move.from_uci('d4c2'), Move.from_uci('b3c2'), Move.from_uci('c3c2'), Move.from_uci('b1c1'), Move.from_uci('c2c1'), Move.from_uci('e1c1')